# naive word2vec

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from scipy import spatial

# Import & Init jieba
import jieba
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')

# Import pandas
import pandas as pd
from pandas import Series, DataFrame

# Import util
import time
import re

### Load datasets

In [ ]:
sample = pd.read_csv('datas/sample_test_data.txt')
x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.dialogue.values]
x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.options.values]
y = sample.answer.values
assert(np.sum([len(_)!=6 for _ in x2]) == 0)

test_datas = pd.read_csv('datas/AIFirstProblem.txt')
test_x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.dialogue.values]
test_x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in test_datas.options.values]
assert(np.sum([len(_)!=6 for _ in test_x2]) == 0)

### word2vec model

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format('models/word2vec/fine-tuned.txt', binary=False)
len(word_vectors.vocab)

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format('models/word2vec/vec200_win40_iter15_mincnt5.bin', binary=True)
len(word_vectors.vocab)

In [ ]:
# Count total occurance to estimate word probability
total_word_cnt = np.sum([_.count for _ in word_vectors.vocab.values()])
total_word_cnt

In [ ]:
unk_cnt = 0
for a, b in zip(x1, x2):
    a_sentence = [word for s in a for word in jieba.cut(s) if word.strip() != '']
    b_sentences = [[word for word in jieba.cut(s) if word.strip() != ''] for s in b]
    
    unk_cnt += len([w for w in a_sentence if w not in word_vectors.vocab])
    unk_cnt += len([w for s in b_sentences for w in s if w not in word_vectors.vocab])
unk_cnt

### Naive trial - centroid

In [ ]:
def centroid(sentence):
    _ = [word_vectors.word_vec(word) for word in sentence if word in word_vectors.vocab]
    return np.mean(_, axis=0) if len(_) > 0 else np.zeros(word_vectors.vector_size)

In [ ]:
def attack_naive_centroid(x1, x2):
    my_cos_ans = []
    my_dot_ans = []
    for a, b in zip(x1, x2):
        a_sentence = [word for s in a for word in jieba.cut(s) if word.strip() != '']
        b_sentences = [[word for word in jieba.cut(s) if word.strip() != ''] for s in b]

        a_center = centroid(a_sentence)
        b_centers = [centroid(s) for s in b_sentences]

        score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
        my_cos_ans.append(np.argmin(score))
        
        score = [np.dot(a_center, bc) for bc in b_centers]
        my_dot_ans.append(np.argmax(score))
    return np.array(my_cos_ans), np.array(my_dot_ans)

In [ ]:
_, __ = attack_naive_centroid(x1, x2)

correct = np.sum(_ == y)
print('%20s: %3d / %3d' % ('centroid (cos)', correct, len(y)))

correct = np.sum(__ == y)
print('%20s: %3d / %3d' % ('centroid (dot)', correct, len(y)))


# Prepare for bagging
# sample_ans.append(_)
# sample_correct.append(correct)
# test_ans.append(attack_naive_centroid(test_x1, test_x2))

### Naive trial - weighted centroid

In [ ]:
def weighted_centroid(sentence, a=1e-4):
    _ = [a / (a + word_vectors.vocab[word].count / total_word_cnt) * word_vectors.word_vec(word)
            for word in sentence if word in word_vectors.vocab]
    return np.mean(_, axis=0) if len(_) > 0 else np.zeros(word_vectors.vector_size)

In [ ]:
def attack_naive_weighted_centroid(x1, x2):
    my_cos_ans = []
    my_dot_ans = []
    for a, b in zip(x1, x2):
        a_sentence = [word for s in a for word in jieba.cut(s)]
        b_sentences = [[word for word in jieba.cut(s)] for s in b]

        a_center = weighted_centroid(a_sentence)
        b_centers = [weighted_centroid(s) for s in b_sentences]

        score = [spatial.distance.cosine(a_center, bc) for bc in b_centers]
        my_cos_ans.append(np.argmin(score))
        
        score = [np.dot(a_center, bc) for bc in b_centers]
        my_dot_ans.append(np.argmax(score))
    return np.array(my_cos_ans), np.array(my_dot_ans)

In [ ]:
_, __ = attack_naive_weighted_centroid(x1, x2)

correct = np.sum(_ == y)
print('%20s: %3d / %3d' % ('weighted centroid (cos)', correct, len(y)))

correct = np.sum(__ == y)
print('%20s: %3d / %3d' % ('weighted centroid (dot)', correct, len(y)))


# Prepare for bagging
# sample_ans.append(_)
# sample_correct.append(correct)
# test_ans.append(attack_naive_weighted_centroid(test_x1, test_x2))

### Naive trial - cosine similarity between two sets of words

In [ ]:
def cos_sim(a_words_set, b_words_set):
    a = [word for word in a_words_set if word in word_vectors.vocab]
    b = [word for word in b_words_set if word in word_vectors.vocab]
    if len(a) == 0 or len(b) == 0:
        return 0
    return word_vectors.n_similarity(a, b)

In [ ]:
def attack_naive_cos_sim(x1, x2):
    my_ans = []
    for a, b in zip(x1, x2):
        a_sentence = [word for s in a for word in jieba.cut(s)]
        b_sentences = [[word for word in jieba.cut(s)] for s in b]

        score = [cos_sim(a_sentence, bs) for bs in b_sentences]
        my_ans.append(np.argmax(score))
    return np.array(my_ans)

In [ ]:
_ = attack_naive_cos_sim(x1, x2)
correct = np.sum(_ == y)
print('%20s: %3d / %3d' % ('cos similarity', correct, len(y)))

## Try bagging

In [ ]:
# QwQ

## Output answer on testing datas

In [ ]:
my_test_ans = attack_naive_cos_sim(test_x1, test_x2)

In [ ]:
with open('answer/attack-naive-word2vec-QAQ.txt', 'w') as fo:
    fo.write('id,ans\n')
    fo.write('\n'.join(['%d,%s' % (i+1, ans) for i, ans in enumerate(my_test_ans)]))
    fo.write('\n')